In [110]:
import torch
import pickle
import numpy as np
import pandas as pd
from copy import deepcopy
from torch import nn
from torch.utils.data import Dataset,DataLoader


In [111]:
def sauvegardeFichier(nomFichier,obj):
    with open(nomFichier,'wb') as fichier :
        enregistre = pickle.Pickler(fichier)
        enregistre.dump(obj)

def lectureFichier(nomFichier):
    with open(nomFichier,'rb') as fichier :
        recupere = pickle.Unpickler(fichier)
        return recupere.load()

In [263]:
class DatasetLd(Dataset):
    def __init__(self,path='../data/LD2011_2014/LD2011_2014.pkl',DIM_T=192,DIM_H=96):
        self.data = lectureFichier('../data/LD2011_2014/LD2011_2014.pkl')
        self.DIM_T = DIM_T
        self.DIM_H = DIM_H

    def __len__(self):
        return len(self.data)

    def __getitem__(self, indice):
        X = self.data[indice:indice+self.DIM_T,:]
        y = self.data[indice+self.DIM_T:indice+self.DIM_T+self.DIM_H,:]
        return (X,y)
    
    def __collate_fn__(self,batch):
        X = torch.stack([ torch.tensor(item[0],dtype=torch.float32) for item in batch])
        y = torch.stack([ torch.tensor(item[1],dtype=torch.float32) for item in batch]) 
        return X,y



    

In [264]:
BATCH_SIZE = 64
DIM_T = 192 # Longeur d'une serie chronologique stage 1
DIM_H = [96, 192, 336, 720]# Nombre de valeur à prédire pour une serie chronologique

dataset1  = DatasetLd(path='../data/LD2011_2014/LD2011_2014.pkl',DIM_T=DIM_T,DIM_H=DIM_H[0])
dataloader1 = DataLoader(dataset1, batch_size=BATCH_SIZE, shuffle=False,collate_fn= dataset1.__collate_fn__,drop_last=True)
dataloader1

# STAGE 1

In [265]:
class Encoder(nn.Module):
    def __init__(self,dim_in) :
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv1d(in_channels=dim_in, out_channels=128, kernel_size=4, stride=2,padding=1),
            nn.Tanh(),
            nn.Dropout(p=0.1,inplace=True),
            nn.Conv1d(in_channels=128, out_channels=64, kernel_size=4, stride=2,padding=1),
            nn.Tanh(),
            nn.Dropout(p=0.1,inplace=True),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=5, stride=3,padding=1),
            nn.Tanh()
        )
        
    def forward(self,x) :
        return self.network(x)
        

class Decoder(nn.Module):
    def __init__(self,dim_out) :
        super().__init__()
        self.network = nn.Sequential(
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=5, stride=3,padding=1),
            nn.Tanh(),
            nn.Dropout(p=0.1,inplace=True),
            nn.ConvTranspose1d(in_channels=64, out_channels=128, kernel_size=4, stride=2,padding=1),
            nn.Tanh(),
            nn.Dropout(p=0.1,inplace=True),
            nn.ConvTranspose1d(in_channels=128, out_channels=dim_out, kernel_size=4, stride=2,padding=1),
            nn.Tanh()
        )
        
    def forward(self,x) :
        return self.network(x)

class Discriminator(nn.Module):
    def __init__(self,dim_in) :
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv1d(in_channels=dim_in, out_channels=128, kernel_size=4, stride=2,padding=1),
            nn.Tanh(),
            nn.Dropout(p=0.1,inplace=True),
            nn.Conv1d(in_channels=128, out_channels=64, kernel_size=4, stride=2,padding=1),
            nn.Tanh(),
            nn.Dropout(p=0.1,inplace=True),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=5, stride=3,padding=1),
            nn.Tanh()
            
        )
        
        self.fc = nn.Sequential(
            nn.Linear(in_features=64, out_features=64),
            nn.Sigmoid()
        )
        
    def forward(self,x) :
        x = self.network(x)
        x = torch.movedim(x,1,2)
        x = self.fc(x)
        x = torch.movedim(x,1,2)
        x = np.where(x >= 0.5,1.,0.)
        return torch.tensor(x,dtype=torch.float32)
    



In [259]:
def measure_similarity(H,M):
    H = H.detach().numpy()
    M = M.detach().numpy()
   
    tmp = np.exp(-1 * (-2 * (M.T@H) + np.sum(M**2,axis=0)[:, np.newaxis]  + np.sum(H**2,axis=1)[:, np.newaxis]))
    sum_tmp = np.sum(tmp,axis=0)
    C = tmp / sum_tmp  
    C = np.where(C.T == np.max(C.T,axis=0),1.,0.).T
    return torch.tensor(C,dtype=torch.float32) # DIM_M * DIM_T2


def calcule_loss_reconstruction(DIM_N,DIM_T,DIM_C,X,X_reconstruit):
    loss =  ( 1 / (DIM_N * DIM_T * DIM_C) ) * torch.sum(torch.pow(X_reconstruit - X,2)) # cout loss reconstruction
    return loss.clone().detach().requires_grad_(True) # torch.tensor(loss,requires_grad=True) 



def calcule_loss_m(DIM_N,DIM_T2,DIM_D,X,H,M):
    H2 = H.clone().detach() # avec stop gradient
    H2.requires_grad = False
    Z = np.argmin(H.detach().numpy()-M.detach().numpy(),axis=1)
    Z2 = np.argmin(H2-M,axis=1) # avec stop gradient
    Z2.requires_grad = False
    loss = ( 1 / (DIM_N * DIM_T2 * DIM_D) ) * torch.sum(torch.sum( torch.pow(H2-Z,2) + torch.pow(H-Z2,2)  ))
    return loss.clone().detach().requires_grad_(True) # torch.tensor(loss,requires_grad=True) 

  
def calcule_loss_d(DIM_N,DIM_T2,DATA_D,DATA_D_reconstruit) :
    torch_zero = torch.zeros_like(DATA_D)
    loss = ( 1 / (DIM_N * DIM_T2) ) * torch.sum(torch.sum(torch.maximum(torch_zero,1-DATA_D) + torch.maximum(torch_zero,1+DATA_D_reconstruit)))
    return loss.clone().detach().requires_grad_(True) # torch.tensor(loss,requires_grad=True) 


def calcule_loss(loss_REC,loss_M,DATA_D_reconstruit,DIM_N,DIM_T2,lambd,gamma):
    loss = loss_REC + lambd * loss_M - (gamma / (DIM_N * DIM_T2) ) * torch.sum(DATA_D_reconstruit)
    return loss.clone().detach().requires_grad_(True) # torch.tensor(loss,requires_grad=True) 


In [266]:

def calcule_mse(DIM_N,DIM_H,DIM_C,y,y_pred):
    return ( 1 / DIM_N * DIM_H * DIM_C ) * torch.nn.functional.mse_loss(y_pred,y)

def calcule_mae(DIM_N,DIM_H,DIM_C,y,y_pred):
    return ( 1 / DIM_N * DIM_H * DIM_C ) * torch.nn.functional.l1_loss(y_pred,y)

In [261]:
# Stage 1


In [268]:
# mse = calcule_mse(DIM_N,DIM_H,DIM_C,y,y_pred)
# mae = calcule_mae(DIM_N,DIM_H,DIM_C,y,y_pred)

In [269]:
def train_stage_1(dataloader,epochs) :
    
    DIM_C = dataloader.dataset.data.shape[1] # Nombre de variable d'une serie chronologique
    DIM_D = 64 # Nombre de variable d'une serie chronologique apres encodeur ( taille couche sortie encodeur)
    DIM_N = len(dataloader.dataset.data) - dataloader.dataset.DIM_T - dataloader.dataset.DIM_H + 1 # Nombre de segment de longeur T
    DIM_M = 16 # Taille de la banque de mémoire ( voir papier taille 16)
    lambd = 0.5
    gamma = 0.5
     
    encoder = Encoder(DIM_C)
    decoder = Decoder(DIM_C)
    discriminator = Discriminator(DIM_C)
    
    opt_encoder = torch.optim.Adam(encoder.parameters(), lr=0.0001)
    opt_decoder = torch.optim.Adam(decoder.parameters(), lr=0.0001)
    opt_discriminator = torch.optim.Adam(discriminator.parameters(), lr=0.0001)
    
    DATA_M = torch.rand((DIM_D,DIM_M),dtype=torch.float32)# Banque de mémoire :  DIM_D X DIM_M
    
    for epoch in epochs :
        for e,(X,Y) in enumerate(dataloader) :
            
            opt_encoder.zero_grad()
            opt_decoder.zero_grad()
            opt_discriminator.zero_grad()
            
            # (1)             
            DATA_X = torch.movedim(X,1,2) # Data : BATCH_SIZE X DIM_C X DIM_T
            DATA_H = encoder(DATA_X) # Data Encoder : BATCH_SIZE X DIM_D X DIM_T2
            DATA_C = measure_similarity(DATA_H,DATA_M) # Matrice similarity : BATCH_SIZE * DIM_M * DIM_T2
            DIM_T2 =  DATA_H.shape[2] # Longeur d'une serie chronologique apres convolution T2 < T à cause convolution et stride suprérieur à 1
            
            DATA_H_reconstruit = (DATA_M @ DATA_C) # Data Encoder : BATCH_SIZE X DIM_D X DIM_T2
            DATA_X_reconstruit = decoder(DATA_H_reconstruit) # Data reconstruit : BATCH_SIZE X DIM_C X DIM_T
            loss_REC = calcule_loss_reconstruction(DIM_N,DIM_T,DIM_C,DATA_X.clone().detach(),DATA_X_reconstruit.clone().detach()) # cout loss reconstruction
            loss_M =  calcule_loss_m(DIM_N,DIM_T2,DIM_D,DATA_X,DATA_H,DATA_M) # cout loss m
            DATA_D = discriminator(DATA_X)  # : BATCH_SIZE X DIM_D X DIM_T2
            DATA_D_reconstruit = discriminator(DATA_X_reconstruit) #  : BATCH_SIZE X DIM_D X DIM_T2
            
            if e % 2 == 0 :
                # (3)                
                loss_D = calcule_loss_d(DIM_N,DIM_T2,DATA_D,DATA_D_reconstruit)
                loss_D.backward()
                print('loss_D : ', loss_D)
            else :
                # (4)  
                loss = calcule_loss(loss_REC,loss_M,DATA_D_reconstruit,DIM_N,DIM_T2,lambd,gamma)
                loss.backward()
                print('loss : ', loss)

#             print('loss_REC : ',loss_REC)
#             print('loss_M : ', loss_M)
#             print('loss_D : ', loss_D)
            
            

            opt_encoder.step()
            opt_decoder.step()
            opt_discriminator.step()
                
    return encoder,decoder, DATA_M
             

In [122]:
encoder,decoder, DATA_M = train_stage_1(dataloader1,epochs=range(1))

loss_D :  tensor(0.0578, requires_grad=True)
loss :  tensor(1088.4798, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0576, requires_grad=True)
loss :  tensor(1184.0475, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0575, requires_grad=True)
loss :  tensor(1213.0448, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0575, requires_grad=True)
loss :  tensor(1209.7692, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0575, requires_grad=True)
loss :  tensor(1189.1252, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0576, requires_grad=True)
loss :  tensor(1132.5608, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0575, requires_grad=True)
loss :  tensor(1164.1729, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0575, requires_grad=True)
loss :  tensor(1161.1810, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0576, requires_grad=True)
loss :  tensor(1125.0885, dtype=torch.float64, requires_gra

loss :  tensor(1334.2545, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0576, requires_grad=True)
loss :  tensor(1234.5281, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0576, requires_grad=True)
loss :  tensor(1261.9625, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0576, requires_grad=True)
loss :  tensor(1333.5301, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0575, requires_grad=True)
loss :  tensor(1381.8652, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0576, requires_grad=True)
loss :  tensor(1327.4462, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0575, requires_grad=True)
loss :  tensor(1302.8488, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0575, requires_grad=True)
loss :  tensor(1318.2280, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0575, requires_grad=True)
loss :  tensor(1281.7570, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0575, requires_gra

loss_D :  tensor(0.0575, requires_grad=True)
loss :  tensor(1325.6861, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0576, requires_grad=True)
loss :  tensor(1390.5224, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0576, requires_grad=True)
loss :  tensor(1509.9306, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0575, requires_grad=True)
loss :  tensor(1502.4437, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0576, requires_grad=True)
loss :  tensor(1596.7974, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0576, requires_grad=True)
loss :  tensor(1636.2016, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0575, requires_grad=True)
loss :  tensor(1783.8886, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0576, requires_grad=True)
loss :  tensor(1832.6164, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0577, requires_grad=True)
loss :  tensor(1905.4628, dtype=torch.float64, requires_gra

loss :  tensor(1161.2602, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0575, requires_grad=True)
loss :  tensor(1091.6111, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0573, requires_grad=True)
loss :  tensor(1062.0019, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0574, requires_grad=True)
loss :  tensor(1063.8410, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0573, requires_grad=True)
loss :  tensor(1054.9708, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0574, requires_grad=True)
loss :  tensor(1063.4550, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0576, requires_grad=True)
loss :  tensor(1063.2190, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0574, requires_grad=True)
loss :  tensor(1064.4288, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0575, requires_grad=True)
loss :  tensor(1035.7482, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0574, requires_gra

loss_D :  tensor(0.0579, requires_grad=True)
loss :  tensor(2820.8259, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0580, requires_grad=True)
loss :  tensor(2829.0116, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0579, requires_grad=True)
loss :  tensor(2734.6553, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0578, requires_grad=True)
loss :  tensor(2641.1228, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0577, requires_grad=True)
loss :  tensor(2536.1367, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0578, requires_grad=True)
loss :  tensor(2443.3522, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0580, requires_grad=True)
loss :  tensor(2662.6320, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0579, requires_grad=True)
loss :  tensor(2723.6000, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0579, requires_grad=True)
loss :  tensor(2686.0446, dtype=torch.float64, requires_gra

...

loss_D :  tensor(0.0588, requires_grad=True)
loss :  tensor(3229.7061, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0585, requires_grad=True)
loss :  tensor(3153.9215, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0583, requires_grad=True)
loss :  tensor(3035.9172, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0585, requires_grad=True)
loss :  tensor(3115.5341, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0588, requires_grad=True)
loss :  tensor(3120.4251, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0588, requires_grad=True)
loss :  tensor(3162.2130, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0588, requires_grad=True)
loss :  tensor(3259.7615, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0587, requires_grad=True)
loss :  tensor(3234.5454, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0586, requires_grad=True)
loss :  tensor(2348.7530, dtype=torch.float64, requires_gra

loss :  tensor(2845.7335, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0588, requires_grad=True)
loss :  tensor(2761.0874, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0588, requires_grad=True)
loss :  tensor(2599.0264, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0587, requires_grad=True)
loss :  tensor(2506.5618, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0587, requires_grad=True)
loss :  tensor(2424.9384, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0587, requires_grad=True)
loss :  tensor(2536.7626, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0587, requires_grad=True)
loss :  tensor(2570.5851, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0586, requires_grad=True)
loss :  tensor(2576.5606, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0586, requires_grad=True)
loss :  tensor(2611.1660, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0585, requires_gra

loss_D :  tensor(0.0582, requires_grad=True)
loss :  tensor(8228.3291, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0582, requires_grad=True)
loss :  tensor(8431.3963, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0582, requires_grad=True)
loss :  tensor(7179.8181, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0581, requires_grad=True)
loss :  tensor(6754.4532, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0581, requires_grad=True)
loss :  tensor(6961.5736, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0581, requires_grad=True)
loss :  tensor(8830.4648, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0582, requires_grad=True)
loss :  tensor(8871.3680, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0582, requires_grad=True)
loss :  tensor(7581.0513, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0581, requires_grad=True)
loss :  tensor(10002.0591, dtype=torch.float64, requires_gr

loss_D :  tensor(0.0581, requires_grad=True)
loss :  tensor(9082.7068, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0582, requires_grad=True)
loss :  tensor(8051.3680, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0582, requires_grad=True)
loss :  tensor(6165.3398, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0582, requires_grad=True)
loss :  tensor(6630.0323, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0581, requires_grad=True)
loss :  tensor(7417.5142, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0580, requires_grad=True)
loss :  tensor(7138.6284, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0583, requires_grad=True)
loss :  tensor(6374.3946, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0583, requires_grad=True)
loss :  tensor(4860.8974, dtype=torch.float64, requires_grad=True)
loss_D :  tensor(0.0583, requires_grad=True)
loss :  tensor(3829.2002, dtype=torch.float64, requires_gra

RuntimeError: stack expects each tensor to be equal size, but got [96, 370] at entry 0 and [95, 370] at entry 1

# STAGE 2

In [327]:
class Predictor(nn.Module):
    def __init__(self,dim_in) :
        super().__init__()
        self.network = nn.Sequential(
            torch.nn.LSTM(input_size=dim_in , hidden_size=128, num_layers=8, dropout=0.5 )
        )
      
    def forward(self,x) :
        x,_ = self.network(x)
        x = torch.nn.Softmax()(x)
        return torch.tensor(x,dtype=torch.float32)

In [328]:
def calcule_loss_pred(DIM_T2,DIM_HH,DATA_C_reconstruit,DATA_CC):
    loss = (1 / (DIM_T2 + DIM_HH)) * torch.nn.functional.binary_cross_entropy(DATA_CC, DATA_C_reconstruit)
    return loss

In [329]:
BATCH_SIZE = 64
DIM_TT = 96 # Longeur d'une serie chronologique stage 2
DIM_H = [96, 192, 336, 720]# Nombre de valeur à prédire pour une serie chronologique

dataset2  = DatasetLd(path='../data/LD2011_2014/LD2011_2014.pkl',DIM_T=DIM_TT,DIM_H=DIM_H[0])
dataloader2 = DataLoader(dataset2, batch_size=BATCH_SIZE, shuffle=False,collate_fn= dataset2.__collate_fn__)
dataloader2

In [330]:
DIM_C = dataloader2.dataset.data.shape[1] # Nombre de variable d'une serie chronologique
encoder = Encoder(DIM_C)
decoder = Decoder(DIM_C)
discriminator = Discriminator(DIM_C)
DATA_M = torch.rand((64,16),dtype=torch.float32)# Banque de mémoire :  DIM_D X DIM_M
    
list_model = [encoder,decoder,discriminator]
for model in list_model :
    for param in model.parameters() :
        param.requires_grad = False

In [331]:
from math import ceil

def train_stage_2(encoder,decoder,DATA_M,dataloader,epochs) :
    
    DIM_C = dataloader.dataset.data.shape[1] # Nombre de variable d'une serie chronologique
    DIM_D = 64 # Nombre de variable d'une serie chronologique apres encodeur ( taille couche sortie encodeur)
    DIM_N = len(dataloader.dataset.data) - dataloader.dataset.DIM_T - dataloader.dataset.DIM_H + 1 # Nombre de segment de longeur T
    DIM_M = 16 # Taille de la banque de mémoire ( voir papier taille 16)

     
    predictor = Predictor(DIM_M)

    opt_predictor = torch.optim.AdamW(predictor.parameters(), lr=0.0001)

    for epoch in epochs :
        for X,Y in dataloader :
         
            opt_predictor.zero_grad()
            
            # (1)             
            DATA_X = torch.movedim(X,1,2) # Data : BATCH_SIZE X DIM_C X DIM_T
            DATA_H = encoder(DATA_X) # Data Encoder : BATCH_SIZE X DIM_D X DIM_T2
            DATA_C = measure_similarity(DATA_H,DATA_M) # Matrice similarity : BATCH_SIZE * DIM_M * DIM_T2
            DIM_T2 =  DATA_H.shape[2] # Longeur d'une serie chronologique apres convolution T2 < T à cause convolution et stride suprérieur à 1
            
            
            # (5)            
            DIM_CC = ceil(DIM_T2*(dataloader.dataset.DIM_T + dataloader.dataset.DIM_H) / dataloader.dataset.DIM_T)
            DIM_HH = ceil(DIM_T2*dataloader.dataset.DIM_H / dataloader.dataset.DIM_T)
        
            DATA_C = torch.movedim(DATA_C,1,2) # Data : BATCH_SIZE X DIM_T2 X DIM_M
            DATA_C_reconstruit = predictor(DATA_C)
            
            # (6) 
            DATA_X_Y = torch.cat((X,Y),dim=0)
            DATA_X_Y = encoder(DATA_X_Y)
            DATA_CC = DATA_M @ DATA_X_Y
            
            # (7) 
            loss_P = calcule_loss_pred(DIM_T2,DIM_HH,DATA_C_reconstruit,DATA_CC)
            loss_P.backward()
        
            print('loss_P : ',loss_P)
            print()

            opt_predictor.step()
                
    return predictor
             

In [332]:
train_stage_2(encoder,decoder,DATA_M,dataloader2,epochs=range(1))

torch.Size([64, 8, 128])


<ipython-input-327-287f5368d715>:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = torch.nn.Softmax()(x)
<ipython-input-327-287f5368d715>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(x,dtype=torch.float32)


RuntimeError: Given groups=1, weight of size [128, 370, 4], expected input[128, 96, 370] to have 370 channels, but got 96 channels instead

# STAGE 3

In [ ]:
def inference(X,encoder,decoder,DATA_M) :
    pass